# Rhasspy MQTT Tools

In [2]:
! which mosquitto_pub

/usr/bin/mosquitto_pub


In [4]:
! mosquitto_pub -t 'test' -m 'test' && echo 'OK'

OK


In [75]:
import os
import signal
import subprocess
from pathlib import Path

profile_dir = Path(os.getcwd()) / "profile"
os.putenv("profile_dir", str(profile_dir))
profile_dir

PosixPath('/home/hansenm/opt/rhasspy-services/docs/notebooks/profile')

In [100]:
%%file profile/profile.yml
speech-to-text:
    audio-input:
        host: 127.0.0.1
        port: 12200
    pocketsphinx:
        acoustic-model: !env "${profile_dir}/acoustic_model"
        language-model: !env "${profile_dir}/language_model.txt"
        dictionary: !env "${profile_dir}/dictionary.txt"
            
intent-recognition:
    fsticuffs:
        intent-fst: !env "${profile_dir}/intent.fst"
        skip-unknown: true
        fuzzy: true

Overwriting profile/profile.yml


In [94]:
def start(*args):
    """Start a process in a new process group"""
    return subprocess.Popen(args, start_new_session=True)

def stop(proc):
    """Terminate a process group"""
    if proc.poll() is None:
        pgrp = os.getpgid(proc.pid)
        os.killpg(pgrp, signal.SIGINT)
        print(f"Terminated {pgrp}")

In [95]:
import io
import jsonlines
import paho.mqtt.client as mqtt

def subscribe_to(*topics):
    """Return an on_connect function that subscribes to MQTT topics."""
    def on_connect(client, userdata, flags, rc):
        for topic in topics:
            client.subscribe(topic)
            print(f"Subscribed to {topic}")
            
    return on_connect

def print_message(client, userdata, msg):
    """MQTT message handler that prints topic/payload."""
    print(msg.topic, msg.payload.decode())
    
def send_json(client, topic, payload_dict):
    """Sends an MQTT message with a jsonl payload."""
    with io.BytesIO() as buffer:
        with jsonlines.Writer(buffer) as out:
            out.write(payload_dict)
            
        client.publish(topic, buffer.getvalue().strip())

---

In [96]:
# Start speech to text MQTT client with profile
rhasspy_stt = start(
    "rhasspy-pocketsphinx-mqtt", "--profile", profile_dir
)

print("Started speech to text")

Started speech to text


In [97]:
# Start MQTT client, subscribe to output event
client = mqtt.Client()
client.on_connect = subscribe_to("rhasspy/speech-to-text/text-captured")
client.on_message = print_message
client.connect("localhost")
client.loop_start()
print("Started MQTT")

Started MQTT
Subscribed to rhasspy/speech-to-text/text-captured


In [98]:
# Start listening
send_json(client, "rhasspy/speech-to-text/start-listening", {})

# Stream audio to udp socket (16-bit 16Khz mono PCM)
! gst-launch-1.0 -q \
    filesrc location=wav/turn_on_living_room_lamp.wav ! \
    decodebin ! \
    audioconvert ! \
    audioresample ! \
    audio/x-raw, rate=16000, channels=1, format=S16LE ! \
    udpsink host='127.0.0.1' port=12202

# Stop listening and transcribe
send_json(client, "rhasspy/speech-to-text/stop-listening", {})

rhasspy/speech-to-text/text-captured {"text": "", "transcribe_seconds": 0.00010085105895996094, "likelihood": 0.0}


In [99]:
# Stop MQTT client
client.loop_stop()
print("Stopped MQTT")

# Stop speech to text client
stop(rhasspy_stt)
print("Stopped speech to text")

Stopped MQTT
Terminated 9812
Stopped speech to text


---

In [104]:
# Start intent recognizer MQTT client with profile
rhasspy_intent = start(
    "rhasspy-fsticuffs-mqtt", "--profile", profile_dir
)

print("Started intent recognizer")

Started intent recognizer


In [102]:
# Start MQTT client, subscribe to output event
client = mqtt.Client()
client.on_connect = subscribe_to("rhasspy/intent-recognition/intent-recognized")
client.on_message = print_message
client.connect("localhost")
client.loop_start()
print("Started MQTT")

Started MQTT
Subscribed to rhasspy/intent-recognition/intent-recognized


In [106]:
send_json(
    client,
    "rhasspy/intent-recognition/recognize-intent",
    { "text": "turn on the living room lamp" }
)

rhasspy/intent-recognition/intent-recognized {"text": "turn on the living room lamp", "intent": {"name": "ChangeLightState", "confidence": 1.0}, "entities": [{"entity": "state", "value": "on", "raw_value": "on", "start": 5, "end": 7}, {"entity": "name", "value": "living room lamp", "raw_value": "living room lamp", "start": 12, "end": 28}], "raw_text": "turn on the living room lamp", "tokens": ["turn", "on", "the", "living", "room", "lamp"], "raw_tokens": ["turn", "on", "the", "living", "room", "lamp"], "slots": {"state": "on", "name": "living room lamp"}, "intents": [], "recognize_seconds": 0.0003459453582763672}


In [107]:
# Stop MQTT client
client.loop_stop()
print("Stopped MQTT")

# Stop intent recognition client client
stop(rhasspy_intent)
print("Stopped intent recognizer")

Stopped MQTT
Terminated 10179
Stopped intent recognizer
